In [86]:
import numpy as np
import pandas as pd
import os
import nibabel as nib
import glob
import math
from nilearn.image import resample_img
from nilearn import plotting
get_ipython().magic(u'matplotlib inline')
import statistics
import math
import matplotlib.pyplot as plt
import time
os.chdir('/projects/hulacon/shared/neu_data/AVFP')


In [87]:
vid_input=0

In [88]:
#get a full list of videoss
logfile_folder = 'AffVidsMem_logfiles'
log_header = ['Video_name', 'video_number', 'video_category(1-H 2-SO 3-SP)', 'novel_vs_familiar(1-N,2-Old)', 'run_number', 'video_onset', 'video_offset', 'video_duration_method1', 'video_duration_method2', 'Fear_rating_onset', 'fear_rating', 'fear_rating_RT', 'Arousal_rating_onset', 'arousal_rating', 'arousal_rating_RT', 'Valence_rating_onset', 'valence_rating', 'valence_rating_RT']
log_files = glob.glob(logfile_folder+'/AffVidsMem*sub*.txt')
log_files.sort()
logs = pd.DataFrame()
for n,s in enumerate(log_files):
    subject_log = pd.read_table(s, sep=' ', header=None, names = log_header, index_col=False)
    subject_log['subject']=s.split('.txt')[0].split('fmri_')[-1]
    logs=logs.append(subject_log)
logs=logs.reset_index(drop=True)
vid_list = list(set(logs['Video_name']))
vid_list.sort()
video_name=vid_list[vid_input]
print(video_name)

Heights_a_1.mp4


In [89]:
#get subject name from log file names
data_folder = '36p'
subj_dir = [i.split('.txt')[0].split('fmri_')[-1] for i in glob.glob(logfile_folder+'/AffVidsMem*sub*.txt')]
subj_dir.sort()

search_size = 5 #edge in voxel size
vox = int((search_size-1)/2) #number of voxels from center voxel
search_size = vox*2+1
print('edge: '+str(search_size)+' voxels')
print('volume: '+str(search_size**3)+' voxels')


edge: 5 voxels
volume: 125 voxels


In [90]:
# log_header = ['Video_name', 'video_number', 'video_category(1-H 2-SO 3-SP)', 'novel_vs_familiar(1-N,2-Old)', 'run_number', 'video_onset', 'video_offset', 'video_duration_method1', 'video_duration_method2', 'Fear_rating_onset', 'fear_rating', 'fear_rating_RT', 'Arousal_rating_onset', 'arousal_rating', 'arousal_rating_RT', 'Valence_rating_onset', 'valence_rating', 'valence_rating_RT']
# num_vid = pd.read_table(glob.glob(os.path.join(logfile_folder,'*'+subj_dir[0]+'.txt'))[0], sep=' ', header=None, names = log_header, index_col=False).shape[0]
#sample subj data img for resampling
subj_data_img = nib.load(glob.glob(os.path.join(data_folder,'sub-'+subj_dir[0][-3:]+'_run1_36p_mc_MNI_masked.nii.gz'))[0])
subj_data = subj_data_img.get_fdata()

TR=0.8
num_TR = 600
TR_series = [x*0.8 for x in range(0, num_TR)]
TR_BOLD_offset = 7
TR_video_duration = 26 #video_duration/0.8=25.6 round to 26
TR_mean_rating= 5.3*3 #rating_duration/0.8=5.3 
TR_mean_mean_ITI = 6 #ITI_duration/0.8=5.8 round to 6
TR_trial_end = TR_video_duration+TR_mean_rating+TR_mean_mean_ITI

#find the index of the closest number to K in a list
def closest(lst, K): 
    val = lst[min(range(len(lst)), key = lambda i: abs(lst[i]-K))] 
    idx = lst.index(val)
    return idx

#use index of finding value x in 2D array (b) of value x as index of the first 2 dimensions for 3D array (a)
def select3Dfrom2D(a,b,x):
    b=old_vid.astype(int)
    
    new_array = []
    for dim1 in list(set(np.where(b==x)[0])):
        new_array.append(a[dim1,np.where(b==x)[1][np.where(b==x)[0]==dim1],:])
    new_array = np.array(new_array)
    
    return new_array


In [91]:
#get grey matter mask
save_dir = '/projects/hulacon/shared/neu_data/AVFP/ISECmem_timecourse_searchlight_correlation_analysis/a'+str(search_size)+'/video/'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)
# mask = '/home/chendanl/roi/HarvardOxford/HarvardOxford-cort-maxprob-thr25-1mm.nii.gz'
# mask = '/home/chendanl/roi/HarvardOxford/HarvardOxford-sub-maxprob-thr25-1mm.nii.gz'
mask = '/home/chendanl/roi/mni_icbm152_gm_tal_nlin_asym_09a_thresh30.nii.gz'
mask_img = nib.load(mask)
mask_img = resample_img(mask_img,
                        target_affine=subj_data_img.affine,
                        target_shape=subj_data_img.shape[0:3],
                        interpolation='nearest')

#get num of voxels
# mask_location = np.where((mask_img.get_fdata()!=0) & (subj_data[:,:,:,0]!=0))
mask_location = np.where(mask_img.get_fdata()!=0)
mask_size = list(range(mask_location[0].shape[0]))

# nib.save(mask_img, '/home/chendanl/wrkdir/roi.nii.gz')
# display = plotting.plot_roi('/home/chendanl/wrkdir/roi.nii.gz', bg_img='/home/chendanl/roi/mni_icbm152_t1_tal_nlin_sym_09b_hires.nii')
# plotting.show()

In [92]:
#loop through each new and old list, 
vid_log = logs[logs['Video_name']==video_name]
vid_log = vid_log.reset_index(drop=True)
old_vid = np.empty(len(vid_log))
old_vid[:] = np.nan
# log_subset_video_old
#load the sub data based on that, 
#calculate the baseline based on that too

#set template for result map
result_template_brain = mask_img.get_fdata().copy()
result_template_brain[:]=np.nan
result_template_brain_new = result_template_brain.copy()
result_template_brain_old = result_template_brain.copy()

#initialize list for averaged acitivty in mask: vid x TR
vid_avg_activity_rest_baseline = np.empty((len(vid_log), TR_video_duration))
vid_avg_activity_rest_baseline[:] = np.nan

print('total number of videos:')
print(vid_log.shape[0])
print('number of new videos:')
print(np.sum(vid_log['novel_vs_familiar(1-N,2-Old)'] == 2))
print('number of old videos:')
print(np.sum(vid_log['novel_vs_familiar(1-N,2-Old)'] == 1))

total number of videos:
51
number of new videos:
26
number of old videos:
25


In [93]:
#see if the array already existed
if os.path.exists(save_dir+video_name.split('.')[0]+'_array.npy'):
    print('reading existing array')
    existing_array = np.load(save_dir+video_name.split('.')[0]+'_array.npy')
    vid_avg_activity_rest_baseline_voxel = existing_array
    vid_range = list(set(np.where(existing_array==0)[0]))
    vid_range.sort()
    if vid_range[0] == 6 and vid_range[1] == 13 and vid_range[2] != 14:
        vid_range=vid_range[2:]
    if vid_avg_activity_rest_baseline_voxel[-1,-1,-1] != 0:
        vid_range=[]
else:
    print('creating new array')
    vid_avg_activity_rest_baseline_voxel = np.empty((vid_log.shape[0], TR_video_duration, len(mask_size)))
    vid_range = range(vid_log.shape[0])
print(vid_range)

reading existing array
[]


In [94]:
#initiate subject data so that don't need to read them over and over again later
for n_vid in vid_range:
    start = time.time()
    print(n_vid)
    subj=vid_log['subject'][n_vid]
    run=vid_log['run_number'][n_vid]

    #get run data
    subj_run_f = glob.glob(os.path.join(data_folder,'sub-'+subj[-3:]+'_run'+str(run)+'_36p_mc_MNI_masked.nii.gz'))[0]
    subj_run_img = nib.load(subj_run_f)
    subj_run_data = subj_run_img.get_fdata()
    subj_run_data[mask_img==0] = np.nan
#     exec("subj_run_data_" + str(n_vid) + " = subj_run_img.get_fdata()")
#     exec("print(subj_run_data_" + str(n_vid) + ".shape)")

    #read subject log file
    subj_log_f = glob.glob(os.path.join(logfile_folder,'AffVidsMem*'+subj+'.txt'))[0]
    subj_log = pd.read_table(subj_log_f, sep=' ', header=None, names = log_header, index_col=False) 

    for mask_coordinate in mask_size:
        print('\rworking on '+str(mask_coordinate) + ' out of '+str(max(mask_size)) + ' voxels, ' +str((mask_coordinate)/max(mask_size)*100) + '% done', end="")

        #get the coordinates of the centered voxel
        x = mask_location[0][mask_coordinate]
        y = mask_location[1][mask_coordinate]
        z = mask_location[2][mask_coordinate]

        #get the range of coordinates based on the edge size vox
        x_min = x-vox; y_min = y-vox; z_min = z-vox
        x_max = x+vox+1; y_max = y+vox+1; z_max = z+vox+1;

        vid_num = subj_log.index[subj_log['Video_name']==video_name][0]

        #calculate ITI period average
        #get averanged run rest data
        rest_mean_run_temp = []
        trial_list = subj_log.index[subj_log['run_number'] == run].tolist()
        for i in trial_list[0:len(trial_list)-1]:
            #add TR offset to find time for ITI period
            subj_run_rest_data = subj_run_data[x_min:x_max,y_min:y_max,z_min:z_max,closest(TR_series, (subj_log['Valence_rating_onset'][i]+TR_BOLD_offset+(TR_mean_rating/3)*TR)): closest(TR_series, subj_log['video_onset'][i+1]+TR_BOLD_offset)]
            subj_run_rest_data[subj_run_rest_data==0.]=np.nan
            rest_mean_run_temp.append(np.nanmean(subj_run_rest_data)) 
        rest_mean_run = np.nanmean(rest_mean_run_temp)

        #get video TR start and end
        TR_begin = closest(TR_series, subj_log['video_onset'][vid_num]+TR_BOLD_offset)
        # TR_mean_trial_duration = math.ceil((subj_log['video_offset'][vid_num] - subj_log['video_onset'][vid_num])/TR)
        TR_end = TR_begin+TR_video_duration

        #get video data from nearest TR start
        vid_data = subj_run_data[x_min:x_max,y_min:y_max,z_min:z_max,TR_begin:TR_end]
        vid_data[vid_data==0.]=np.nan

        #get average of activity in masked area at each slice
        for i in range(vid_data.shape[3]):
            #BOLD signal mean with run1 rest period as baseline
            vid_avg_activity_rest_baseline_voxel[n_vid, i, mask_coordinate] = np.nanmean(vid_data[:,:,:,i])-rest_mean_run
                
    end = time.time()
    print(end - start)
    np.save(save_dir+video_name.split('.')[0]+'_array.npy', vid_avg_activity_rest_baseline_voxel)


In [ ]:
save_dir = '/projects/hulacon/shared/neu_data/AVFP/ISECmem_timecourse_searchlight_correlation_analysis/a'+str(search_size)+'/video/'
vid_avg_activity_rest_baseline_voxel=np.load(save_dir+video_name.split('.')[0]+'_array.npy')

#loop through all voxels
save_dir = '/home/chendanl/wrkdir/ISECmem_timecourse_searchlight_correlation_analysis/a'+str(search_size)+'/'
for mask_coordinate in mask_size:
#     start = time.time()
    print('\rworking on '+str(mask_coordinate) + ' out of '+str(max(mask_size)) + ' voxels, ' +str((mask_coordinate)/max(mask_size)*100) + '% done', end="")
    #get the coordinates of the centered voxel
    x = mask_location[0][mask_coordinate]
    y = mask_location[1][mask_coordinate]
    z = mask_location[2][mask_coordinate]
    
    #loop video
    for n_vid in range(vid_log.shape[0]):
#         print(n_vid)
        subj=vid_log['subject'][n_vid]
        run=vid_log['run_number'][n_vid]

        #get old/new info for each video
        if vid_log['novel_vs_familiar(1-N,2-Old)'][n_vid] == 2:
            old_vid[n_vid] = 1
        elif vid_log['novel_vs_familiar(1-N,2-Old)'][n_vid] == 1:
            old_vid[n_vid] = 0
            
    vid_avg_activity_rest_baseline = vid_avg_activity_rest_baseline_voxel[:,:,mask_coordinate]
            
    #get the old/new video timecourse seperately 
    vid_avg_activity_rest_baseline_old = vid_avg_activity_rest_baseline[old_vid==1, :]
    vid_avg_activity_rest_baseline_new = vid_avg_activity_rest_baseline[old_vid==0, :]

    #get autocorrelation for timecourse for average, new, and old images
    correlation_coe = np.corrcoef(np.transpose(vid_avg_activity_rest_baseline))
    correlation_coe[correlation_coe>=0.9999999999]=np.nan
    correlation_coe_mean = np.nanmean(correlation_coe)
    result_template_brain[x,y,z] = correlation_coe_mean
    result_img = nib.Nifti1Image(result_template_brain, subj_data_img.affine, subj_data_img.header)
    nib.save(result_img, save_dir+video_name.split('.')[0]+'_corr_avg.nii.gz')

    #new
    correlation_coe_new = np.corrcoef(np.transpose(vid_avg_activity_rest_baseline_new))
    correlation_coe_new[correlation_coe_new>=0.9999999999]=np.nan
    correlation_coe_new_mean = np.nanmean(correlation_coe_new)
    result_template_brain_new[x,y,z] = correlation_coe_new_mean
    result_img_new = nib.Nifti1Image(result_template_brain_new, subj_data_img.affine, subj_data_img.header)
    nib.save(result_img_new, save_dir+video_name.split('.')[0]+'_corr_new.nii.gz')

    #old
    correlation_coe_old = np.corrcoef(np.transpose(vid_avg_activity_rest_baseline_old))
    correlation_coe_old[correlation_coe_old>=0.9999999999]=np.nan
    correlation_coe_old_mean = np.nanmean(correlation_coe_old)
    result_template_brain_old[x,y,z] = correlation_coe_old_mean
    result_img_old = nib.Nifti1Image(result_template_brain_old, subj_data_img.affine, subj_data_img.header)
    nib.save(result_img_old, save_dir+video_name.split('.')[0]+'_corr_old.nii.gz')

#     end = time.time()
#     print(end - start)
# # plotting.view_img('/home/chendanl/wrkdir/ISECmem_timecourse_searchlight_correlation_analysis/'+subj+'_corr_diff.nii.gz',cut_coordsNone=[10,38,35])
# plotting.view_img(result_img, save_dir+video_name.split('.')[0]+'_corr_avg.nii.gz')


working on 109 out of 166077 voxels, 0.06563220674747255% done

/packages/miniconda/20190102/envs/anaconda-tensorflow-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in greater_equal
/packages/miniconda/20190102/envs/anaconda-tensorflow-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:35: RuntimeWarning: Mean of empty slice
/packages/miniconda/20190102/envs/anaconda-tensorflow-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in greater_equal
/packages/miniconda/20190102/envs/anaconda-tensorflow-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:43: RuntimeWarning: Mean of empty slice
/packages/miniconda/20190102/envs/anaconda-tensorflow-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:50: RuntimeWarning: invalid value encountered in greater_equal
/packages/miniconda/20190102/envs/anaconda-tensorflow-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:51: RuntimeWarning: Mean of empty slice


working on 163219 out of 166077 voxels, 98.27911149647453% done

In [ ]:
# plotting.view_img('/home/chendanl/wrkdir/ISECmem_timecourse_searchlight_correlation_analysis/a5/Heights_a_1_corr_avg.nii.gz')
